<a href="https://colab.research.google.com/github/veereshthotigar/CSEE5590-490-AI-CyberSecurity-/blob/master/ICP6_add/L5_Encrypted_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

### Step 1: Create Data Owners

In [1]:
!pip install syft

     |████████████████████████████████| 225kB 1.8MB/s 
     |████████████████████████████████| 389kB 35.3MB/s 
     |████████████████████████████████| 450kB 44.4MB/s 
     |████████████████████████████████| 1.4MB 27.1MB/s 
     |████████████████████████████████| 256kB 41.0MB/s 
     |████████████████████████████████| 204kB 41.7MB/s 
     |████████████████████████████████| 81kB 26.5MB/s 
     |████████████████████████████████| 51kB 19.5MB/s 
     |████████████████████████████████| 276kB 42.1MB/s 
     |████████████████████████████████| 122kB 40.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [0]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0709 23:52:43.418392 140194814519168 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0709 23:52:43.446295 140194814519168 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
ted = sy.VirtualWorker(hook, id="ted")

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, ted])
alice.add_workers([bob, ted])
ted.add_workers([alice, bob])

W0709 23:53:13.103533 140194814519168 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:53:13.107202 140194814519168 base.py:628] Worker ted already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:53:13.110024 140194814519168 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:53:13.112899 140194814519168 base.py:628] Worker ted already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:53:13.115076 140194814519168 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:53:13.117540 140194814519168 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:ted #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)


### Step 2: Create the Model

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

### Step 3: Send a Copy of The Model to Alice and Bob

In [0]:
bobs_model = model.copy().send(bob) 
alices_model = model.copy().send(alice) 

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)


### Step 4: Train Bob's and Alice's Models (in parallel)

In [0]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).mean()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    print(f'Bob loss {bobs_loss}')

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).mean()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    print(f'alice loss {alices_loss}')


Bob loss 0.08170386403799057
alice loss 0.8557154536247253
Bob loss 0.05373760685324669
alice loss 0.31454747915267944
Bob loss 0.037807364016771317
alice loss 0.1492169350385666
Bob loss 0.02848769538104534
alice loss 0.09549886733293533
Bob loss 0.022818222641944885
alice loss 0.07520535588264465
Bob loss 0.01918213441967964
alice loss 0.06517563760280609
Bob loss 0.01669507659971714
alice loss 0.058526940643787384
Bob loss 0.014872046187520027
alice loss 0.053194135427474976
Bob loss 0.013445921242237091
alice loss 0.04853921756148338
Bob loss 0.012268505059182644
alice loss 0.04434875771403313


### Step 5: Send Both Updated Models to a Secure Worker

In [0]:
alices_model.move(ted) # the move function iterates on all objects in alice and call send on it. NOTEICE: inline function
bobs_model.move(ted)


In [0]:
ted._objects # can you identify those objects in the Secure worker?

{35233340024: Parameter containing:
 tensor([[-0.0992,  0.2106]], requires_grad=True),
 39225384585: Parameter containing:
 tensor([-0.1183], requires_grad=True),
 62404059758: Parameter containing:
 tensor([[0.3419, 0.3992]], requires_grad=True),
 70928860650: Parameter containing:
 tensor([0.4320], requires_grad=True)}

### Step 6: Average The Models

In [0]:
# A pytorch trick: to set the weights: you have to either access the data[idx] 
# or wrap the set function with (with torch.no_grad():)
# below we illustrate the first way
# the next cell demonstrates the complete way:

model.weight.data[0].set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
model.bias.data[0].set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

tensor([0.1568])

### Step 7: The complete solution over multiple training sessions

In [0]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(ted)
    bobs_model.move(ted)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0260) Alice:tensor(0.0954)
Bob:tensor(0.0067) Alice:tensor(0.0461)
Bob:tensor(0.0012) Alice:tensor(0.0212)
Bob:tensor(0.0004) Alice:tensor(0.0098)
Bob:tensor(0.0007) Alice:tensor(0.0046)
Bob:tensor(0.0012) Alice:tensor(0.0022)
Bob:tensor(0.0014) Alice:tensor(0.0011)
Bob:tensor(0.0014) Alice:tensor(0.0006)
Bob:tensor(0.0013) Alice:tensor(0.0003)
Bob:tensor(0.0011) Alice:tensor(0.0002)


In [0]:
preds = model(data)
loss = ((preds - target) ** 2).mean()

In [0]:
print(preds)
print(target)
print(loss.data)

tensor([[0.1081],
        [0.0960],
        [0.8771],
        [0.8651]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0136)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [0]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [0]:
bob_x_share *= 2
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

20

This even works for addition between two shared values!!

In [0]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted ( you are the dealer, thus still can see --and know everything-- but Alice and Bob did not really know anything about the overall deal --the sum of the numbers; the secrete--)

In this case, however, to keep the secrete is not about the relaiability / integrity of the person who holds it, rather the fact that those shareholders do not know each other!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant.

In [0]:
x = 5

Q = 9923 # the little fix; a very large prime number

bob_x_share = 1525 # give bob a random number.. try with negatives

alice_x_share = Q - bob_x_share + x

print(f'Bob share= {bob_x_share}')
print(f'Alice share= {alice_x_share}')
print(f'Add both shares= {bob_x_share + alice_x_share}... Wrong secrete')

Bob share= 1525
Alice share= 8403
Add both shares= 9928... Wrong secrete


In [0]:
# Decryption happens by summing all the shares together MODULUS some constant

(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so:  `x_share = (2,5,7)`

Even though normally those shares would be distributed among several workers, you can store them in ordered tuples like this for now.

In [0]:
import random

Q = 23740629843760239486723

# accepts a secrete and shares it among n_shahre, returns a tuple with n_share shares
def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

  # accepts a tuple of shares and return the decrypted values
def decrypt(shares):
    return sum(shares) % Q

In [0]:
shares = encrypt(x=7, n_share=10)
shares

(11948676270872283351394,
 8703366766848657753650,
 7965477960572691232500,
 20276516807477849386333,
 11538313535763403723712,
 21311047368881788802275,
 661528832735043905287,
 10535261790964653455099,
 14123304741295953884072,
 11639655143388871939300)

In [0]:
decrypt(shares)

7

In [0]:
def add(a, b):
    c = list()
    assert(len(a) == len(b))
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [0]:
# run this block of code multiple times, what do you notice baout x and y?

x = encrypt(5)
print(x)
y = encrypt(7)
print(y)

z = add(x,y)
decrypt(z)

(4441645295519819944396, 22043341928151511539064, 20996272463849147489991)
(8452578496449999106250, 6447896460833317624504, 8840154886476922755976)


12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE= 10  # integer numbers (normal numbers)
PRECISION= 3  # how many porecisions after the decimal point to encode
Q = 23740629843760239486723 

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [0]:
encode(-0.600)

23740629843760239345664

In [0]:
decode(9323)

9.323

In [0]:
# run this cell multiple times and notice the changes

x = encrypt(encode(5.5))
y = encrypt(encode(5.5))
z = add(x,y)


print(f'Value of x = {x}')
print(f'Value of y = {y}')
print(f'Value of sum = {z}')
print(f'decoded sum = {decode(decrypt(z))}')

Value of x = (3065453432863962446283, 3919952996103072016334, 16755223414793205029606)
Value of y = (3056704016098173129566, 9427043250768801798134, 11256882576893264564523)
Value of sum = (6122157448962135575849, 13346996246871873814468, 4271476147926230107406)
decoded sum = 11.0


# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
# Its a good idea to restrat the environment at this point

import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
ted = sy.VirtualWorker(hook, id="ted")


bob.add_workers([alice, ted])
alice.add_workers([bob, ted])
ted.add_workers([alice, bob])

W0710 00:00:07.911783 140194814519168 hook.py:98] Torch was already hooked... skipping hooking process
W0710 00:00:07.918199 140194814519168 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:00:07.920352 140194814519168 base.py:628] Worker ted already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:00:07.923529 140194814519168 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:00:07.926409 140194814519168 base.py:628] Worker ted already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:00:07.929332 140194814519168 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:00:07.930728 140194814519168 base.py:628] Worker bob already exists. Replacing old worker which cou

<VirtualWorker id:ted #objects:44>

In [0]:
x = th.tensor([1,2,3,4,5]) ##### the following steps work for integer values only

### Secret Sharing Using PySyft

We can share using the simple` .share()` method!

In [0]:
x = x.share(bob, alice, ted) # implemets additive secrete shares
x # it's a pointer to three secrete shares

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:61477190213 -> bob:84555652421]
	-> (Wrapper)>[PointerTensor | me:71026411111 -> alice:19284694165]
	-> (Wrapper)>[PointerTensor | me:30856543992 -> ted:70381051381]
	*crypto provider: me*

In [0]:
bob._objects # take a look at one of the shares.. Large random numbers --encrypted

{12264841642: tensor([[0.],
         [0.]], requires_grad=True), 16604174385: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 74026504627: tensor([[ 0.0321],
         [-0.0103]], grad_fn=<AddmmBackward>), 84555652421: tensor([3467280445042765661,  509106085931112698, 1800020087709809210,
         1492840272511439380, 3390413188523620295])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:58309431292 -> bob:52168024908]
	-> (Wrapper)>[PointerTensor | me:81364098726 -> alice:95408463803]
	-> (Wrapper)>[PointerTensor | me:82177657038 -> ted:80636918700]
	*crypto provider: me*

In [0]:
bob._objects # bob has another tensor.. why?

{12264841642: tensor([[0.],
         [0.]], requires_grad=True), 16604174385: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 52168024908: tensor([6934560890085531322, 1018212171862225396, 3600040175419618420,
         2985680545022878760, 6780826377047240590]), 74026504627: tensor([[ 0.0321],
         [-0.0103]], grad_fn=<AddmmBackward>), 84555652421: tensor([3467280445042765661,  509106085931112698, 1800020087709809210,
         1492840272511439380, 3390413188523620295])}

In [0]:
y.get() # notice that we have doubled the values on each worker!

tensor([ 2,  4,  6,  8, 10])


### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using `.fix_precision()`
This is important when doing FL since the values you are sharing are decimal numbers.

**Q: what values are we sharing? what are the shared secretes?**

In [0]:
x = th.tensor([0.1, 0.2, 0.3, 0.4]) # a decimal tensor!

In [0]:
x

tensor([0.1000, 0.2000, 0.3000, 0.4000])

In [0]:
x = x.fix_prec() # call this function to encode the decimal tensor using fixed precision
x # notice that this is a tensor chain <-------

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400])

In [0]:
print(f'Type of x {type(x)}')
print(f'Type of x.child {type(x.child)}')
print(f'Type of x.child.child {type(x.child.child)}') # <-- this is how you get the data from a fixed precision tensor

Type of x <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>
Type of x.child <class 'syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor'>
Type of x.child.child <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>


In [0]:
# to decode the previous tensor
print(x.float_prec())

tensor([0.1000, 0.2000, 0.3000, 0.4000])


In [0]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([200, 400, 600, 800])

In [0]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000, 0.8000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice)
print(f'Type of x.child.child {type(x.child.child)}')

Type of x.child.child <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>


In [0]:
bob._objects

{12264841642: tensor([[0.],
         [0.]], requires_grad=True), 16604174385: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 74026504627: tensor([[ 0.0321],
         [-0.0103]], grad_fn=<AddmmBackward>), 84555652421: tensor([3467280445042765661,  509106085931112698, 1800020087709809210,
         1492840272511439380, 3390413188523620295]), 91623044749: tensor([2500054353339609915, 4427401764358621835, 2642660911665621786])}

In [0]:
y = x + x

In [0]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

# Final Project: Federated Learning with Encrypted Gradient Aggregation

Reuse your project from the Secuered Agggregator above to train the same model using the FL appraoch, but this time use the Additive Sharing Encryption so that the participating members have access to their own models only. There is no secure aggregator in this senario. 
Include four members in this project (Alice, Bob, Ted, and Carol). 

Hint: we will *share* the model with the shareholders, do addition on the encrypted models, and then aggregate the final model locally. 

Step 1: Import packages

In [2]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)


W0711 22:46:36.731505 140012058609536 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0711 22:46:36.749588 140012058609536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



Step 2: create a couple workers

In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
ted = sy.VirtualWorker(hook, id="ted")
carol = sy.VirtualWorker(hook, id="carol")
crypto_provider = sy.VirtualWorker(id="crypto_provider", hook=hook)

In [102]:
bob.clear_objects()
ted.clear_objects()
alice.clear_objects()
carol.clear_objects()

<VirtualWorker id:carol #objects:0>

In [103]:
# Clear workers if previously holding any values
print(bob._objects)
print(alice._objects)
print(ted._objects)
print(carol._objects)

{}
{}
{}
{}


In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.],[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.],[0],[0],[1],[1.]], requires_grad=True)


In [105]:
# get pointers to training data on each worker by
# sending some training data to bob and alice
data = data.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
target = target.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
print(bob._objects)

{56855261262: tensor([[ 991490757418982939, 1834162455795001558],
        [1744685931027012132, 3471505681120701979],
        [1542693883532751411, 2322949877600644855],
        [4191112562877455842,  155287764986443308],
        [3241185948679933657, 1550930122397711896],
        [4388405645903313149, 4443059156938381288],
        [2849871903967545055, 3300181080634281394],
        [1799511705847196925, 2832642555634045447]]), 68866476837: tensor([[4103452486853479973],
        [2022213130539845440],
        [ 324821719466965901],
        [1231165375808139605],
        [1307074887020533652],
        [3855450332356468153],
        [2675926951104739278],
        [2598579746862861371]])}


Step 3: Create the Model

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)


Step 4: Send a Copy of The Model to workers


In [0]:

model = model.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [108]:
print(model.weight)
print(opt)

Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:75413016205 -> bob:82019026954]
	-> (Wrapper)>[PointerTensor | me:39464161349 -> alice:17516165243]
	-> (Wrapper)>[PointerTensor | me:15775433014 -> ted:74616748764]
	-> (Wrapper)>[PointerTensor | me:13396827956 -> carol:7179244196]
	*crypto provider: crypto_provider*
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)


Step 5: Train the Models (in parallel)


In [110]:
for i in range(100):

    # Train Model'S
    opt.zero_grad()
    pred = model(data)
    loss = ((pred - target)**2).sum()
    loss.backward()

    opt.step()
    loss = loss.get().float_precision()
    print(F'Iter : {i}, loss {loss.item()}')
    if(loss.item() < 10 and loss.item() > 0):
      break


Iter : 0, loss 67160311332864.0
Iter : 1, loss -24107091492864.0
Iter : 2, loss 2338456862720.0
Iter : 3, loss -47047755431936.0
Iter : 4, loss -37750598270976.0
Iter : 5, loss -4945203429376.0
Iter : 6, loss 567569416192.0
Iter : 7, loss -8662886121472.0
Iter : 8, loss 19124992344064.0
Iter : 9, loss -9430566436864.0
Iter : 10, loss -36289025933312.0
Iter : 11, loss -80701705682944.0
Iter : 12, loss -30150540394496.0
Iter : 13, loss 26909519904768.0
Iter : 14, loss 109204725760.0
Iter : 15, loss 40110305312768.0
Iter : 16, loss 10196770357248.0
Iter : 17, loss -39927312023552.0
Iter : 18, loss 45946557693952.0
Iter : 19, loss -12965257412608.0
Iter : 20, loss 2161072930816.0
Iter : 21, loss 1305439240192.0
Iter : 22, loss -46125419593728.0
Iter : 23, loss 6410907680768.0
Iter : 24, loss 3.561000108718872


In [111]:
#models in the workers
model.weight


Parameter containing:
Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:4731031788 -> bob:30809333377]
	-> (Wrapper)>[PointerTensor | me:91698702940 -> alice:64775034742]
	-> (Wrapper)>[PointerTensor | me:91412505541 -> ted:62325704809]
	-> (Wrapper)>[PointerTensor | me:19495220885 -> carol:32250627981]
	*crypto provider: crypto_provider*

In [112]:
#aggregated model 
model = model.get()
print(model.float_precision().weight)

Parameter containing:
tensor([[0.6970, 0.4680]], requires_grad=True)
